## Sales Scraper for crexi.com (Sales)

 A scraper dedicated to extracting information from Loopnet.com
 During running a brower will open, it's important that the browser opens with a proper resoltion so that all element are loaded.

## ⚠️WARNING⚠️

Overuse of this script may cause the provider to refuse the connection, either temporarily or permanently. To ensure safe usage:
- Avoid running the script continuously all day.
- Keep the randSleep function (present in the first cell) unchanged or increase its values for longer pauses.
- Only use when necessary
- This scrapper is kinda slow, and it's intented to be slow, as to avoid ban from the provider


## SETUP

1. install the dependecies
2. put the chromedriver on the drivers folder
3. all done


## USAGE

Run the first cell to do the setup. A browser will open, Do not close the browser.

Go into crexi.com, login with an account, and go into the results of your search, for example for San Antonio - TX



In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
import numpy as np
import random
import json
import pandas as pd

def randSleep():
    time.sleep(random.uniform(5, 7))

def randSleepLong():
    time.sleep(random.uniform(10, 15))   


def pretty_print(d, indent=0):
    for key, value in d.items():
        print("  " * indent + str(key) + ":")
        if isinstance(value, dict):
            pretty_print(value, indent + 1)
        else:
            print("  " * (indent + 1) + str(value))


service = Service("./drivers/chromedriver")
driver = webdriver.Chrome()
driver.page_load_strategy = 'normal'  # Default behavior, waits for complete page load
wait = WebDriverWait(driver, 10) # mistery wait
driver.get("https://google.com/")



With the browser open on the search page, change the results per page to 100. This is done to speed up the link gathering process.

Run the cell below. It will go trought every page, and save the link for each ad on a file ```urls.txt```

In [39]:
def get_cards():
    cards = driver.find_elements(By.CSS_SELECTOR, ".cui-card-cover-link")
    cards_return = []
    for i in range(len(cards)):
        print(i,cards[i].get_attribute("href"))
        cards_return.append(cards[i].get_attribute("href"))
    
    return cards_return

def get_max_page():
    pages = driver.find_elements(By.CSS_SELECTOR, ".pagination.ng-star-inserted")
    pages = just_text(pages, "\n")
    pages = [page for page in pages if page.isdigit()]
    return pages[-1]

first_page = driver.current_url


urls = open("urls.txt", "w")

pages = driver.find_elements(By.CSS_SELECTOR, ".next.ng-star-inserted")
print(len(pages))


nxt_page = driver.find_elements(By.CSS_SELECTOR, ".next.ng-star-inserted")

page = 1


while True:
    print("page: ", page)
    cards = get_cards()
    for j in range(len(cards)):
        urls.write(cards[j] + "\n")
    randSleep()
    #seo-footer ng-star-inserted
    
    nxt_page = driver.find_elements(By.CSS_SELECTOR, ".next.ng-star-inserted")
    if nxt_page == []:
        break
    else:
        nxt_page[0].click()
    randSleepLong()

    page += 1

print("done with link gathering \a")



1
page:  1
0 https://www.crexi.com/lease/properties/624232/texas-park-north-shopping-center?recommId=ff76210a802d5d6bce4b74b231c20130
1 https://www.crexi.com/lease/properties/393618/texas-427-lombrano-st?recommId=ff76210a802d5d6bce4b74b231c20130
2 https://www.crexi.com/lease/properties/445280/texas-the-andy-san-antonio?recommId=ff76210a802d5d6bce4b74b231c20130
3 https://www.crexi.com/lease/properties/647613/texas-1432-austin-highway?recommId=ff76210a802d5d6bce4b74b231c20130
4 https://www.crexi.com/lease/properties/428134/texas-ingram-hills?recommId=ff76210a802d5d6bce4b74b231c20130
5 https://www.crexi.com/lease/properties/298746/texas-la-arcata-retail-center---phase-i-ii-iii?recommId=ff76210a802d5d6bce4b74b231c20130
6 https://www.crexi.com/lease/properties/400076/texas-north-star-square?recommId=ff76210a802d5d6bce4b74b231c20130
7 https://www.crexi.com/lease/properties/450354/texas-west-plaza?recommId=ff76210a802d5d6bce4b74b231c20130
8 https://www.crexi.com/lease/properties/695526/texas-

The cell below is to load a list of links from a file, like the one created from the cell above

In [2]:
link_list = np.loadtxt("urls.txt", dtype=str)
print(link_list.shape)

(840,)


This next cell is to load all the functions that will be used, run it.

In [3]:
#Done part

def just_text(element,splitter): 
    text = ""
    for e in element:
        text  += e.text
    text_array = text.split(splitter)
    return text_array

def use_nav(index): 
    subtitles = driver.find_elements(By.TAG_NAME, "h2")
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", subtitles[1])
    time.sleep(2)
    nav = driver.find_elements(By.CSS_SELECTOR, ".slide-container")
    tags = nav[0].find_elements(By.TAG_NAME, "a")
    tags[index].click()
    time.sleep(2)

def get_adress(): ## OK
    address = driver.find_elements(By.CSS_SELECTOR, ".heading-description")
    address = address[0].text.split("\n")[0]
    return address

def get_listing(): # get the listing infos
    info_box = driver.find_elements(By.CSS_SELECTOR, ".overview-section-content.cui-body-2-text.ng-star-inserted")
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", info_box[0])
    #print(info_box[0].text)
    info_names = info_box[0].find_elements(By.CSS_SELECTOR, ".overview-property-key")
    infos = driver.find_elements(By.CSS_SELECTOR, ".overview-property-value")
    info_type = []
    info_val = []
    for i in range(len(info_names)):
        info_type.append(info_names[i].text.split("\n")[0])
        info_val.append(infos[i].text.split("\n")[0])
    return info_type, info_val

def get_contacts(): # get the contact information
    names = []
    phones = []
    licenses = []
    contacts_boxs = driver.find_elements(By.CSS_SELECTOR, ".info-container")
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", contacts_boxs[0])
    time.sleep(1)
    to_click = contacts_boxs[0].find_elements(By.TAG_NAME, "a")
    for i in range(len(to_click)):
        if to_click[i].text == "Contact":
            to_click[i].click()
    time.sleep(1)
    brokers_box = driver.find_elements(By.CSS_SELECTOR, ".brokers-list")
    brokers = brokers_box[0].find_elements(By.CSS_SELECTOR, ".ng-untouched.ng-pristine.ng-valid.ng-star-inserted")
    for i in range(len(brokers)):
        name = brokers[i].find_elements(By.CSS_SELECTOR, ".name")
        if len(name) == 0:
            name = "-"
        else:
            name = name[0].text.replace("\n", " ")

        phone = brokers[i].find_elements(By.CSS_SELECTOR, ".broker-phone")
        if len(phone) == 0:
            phone = "-"
        else:
            phone = phone[0].text.split("\n")[1]

        license = brokers[i].find_elements(By.CSS_SELECTOR, ".broker-license")
        if len(license) == 0:
            license = "-"
        else:
            license = license[0].text.split(":")[1]
            license = license.replace("License: ", "")
        
        names.append(name)
        phones.append(phone)
        licenses.append(license)
    return names, phones, licenses

def get_details(): # get the information of facts from the property
    overview_box = driver.find_elements(By.ID, "overview")
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", overview_box[0])
    time.sleep(1)
    buttons = overview_box[0].find_elements(By.TAG_NAME, "button")
    #print(len(buttons))
    for i in range(len(buttons)):
        if buttons[i].text == "View More":
            try:
                buttons[i].click()
                time.sleep(2)
            except:
                pass
    detail_box = driver.find_elements(By.CSS_SELECTOR, ".overview-section-content.cui-body-2-text.ng-star-inserted")
    detail_box = detail_box[1]
    details = detail_box.find_elements(By.CSS_SELECTOR, ".overview-property-key")
    details_vals = detail_box.find_elements(By.CSS_SELECTOR, ".overview-property-value")
    details_names = []
    details_values = []
    for i in range(len(details)):
        details_names.append(details[i].text)
        details_values.append(details_vals[i].text)
        #print(details[i].text, details_vals[i].text)
    return details_names, details_values



def get_general_price(): # Check
    price = driver.find_elements(By.CSS_SELECTOR, ".heading-pricing")[0].text.replace("\n", "")
    return price

def get_spaces():
    spaces = {}
    spaces = driver.find_elements(By.TAG_NAME, "crx-space")
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", spaces[0])
    time.sleep(1)
    space_counter = 0
    #print("# of spaces: ",len(spaces))
    for space in spaces:
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", space)
        time.sleep(1)
        temp_space = {}
        expand_button = space.find_elements(By.TAG_NAME, "button")
        expand_button[0].click()
        time.sleep(1)
        space_inf_name = space.find_elements(By.CSS_SELECTOR, ".overview-property-key")
        space_inf_val = space.find_elements(By.CSS_SELECTOR, ".overview-property-value")
        close_button = space.find_elements(By.TAG_NAME, "button")
        close_button[0].click()
        for i in range(len(space_inf_name)):
            temp_space[space_inf_name[i].text] = space_inf_val[i].text
        spaces[space_counter] = temp_space
        space_counter += 1

    return spaces



With the list of links properlly allocated into ```link_list```, run the next cell, and wait.

In [5]:

def get_infos():
    return_infos = {}

    return_infos["link"] = driver.current_url
    return_infos["address"] = get_adress()

    price = get_general_price()
    return_infos["price"] = price

    ad_date_type, ad_date_val = get_listing()
    for i in range(len(ad_date_type)):
        return_infos[ad_date_type[i]] = ad_date_val[i]

    details_names, details_values = get_details()
    for i in range(len(details_names)):
        return_infos[details_names[i]] = details_values[i]

    return_infos["spaces"] = get_spaces()
    
    return_infos["names"], return_infos["phones"], return_infos["licenses"] = get_contacts()

    return return_infos




datajson = {}
i= 0

max_attempts = 3
while i < len(link_list):
    print(i,len(link_list),"link: ", link_list[i])
    attempt = 0
    while attempt < max_attempts:
        try:
            driver.get(link_list[i])
            time.sleep(4)
            datajson[i] = get_infos()
            print(datajson[i])
            i += 1
            break
        except:
            print("Loading page " + str(i) + " failed, trying attempt " + str(attempt))
            attempt += 1

    else:
        print("Max attempts reached, skipping to next page")
        i += 1
    
    with open("data.json", "w") as outfile:
        json.dump(datajson, outfile)
        
    







0 840 link:  https://www.crexi.com/lease/properties/624232/texas-park-north-shopping-center?recommId=ff76210a802d5d6bce4b74b231c20130
{'link': 'https://www.crexi.com/lease/properties/624232/texas-park-north-shopping-center?recommId=ff76210a802d5d6bce4b74b231c20130', 'address': '842 Northwest Loop 410 San Antonio, TX 78216', 'price': 'Undisclosed Rate ( 18 spaces )', 'Date Listed:': 'Apr 6, 2023', 'Last Updated:': 'Dec 16, 2024', 'Property Type:': 'Retail', 'Subtype:': 'Shopping Center, Entertainment, Retail', 'Tenancy:': 'Multiple', 'Total Building SQFT:': '635,370', 'Minimum Divisible SQFT:': '57', 'Max Contiguous SQFT:': '18,375', 'Vacant SQFT:': '105,009', 'Lot Size (sq ft):': '635,370', 'Buildings:': '17', 'County:': 'Bexar County', 'spaces': [{'Space / Unit:': 'L05G', 'Space Subtype:': 'Shopping Center', 'RSF:': '3825', 'Days on Market:': '209 Days', 'Listing Type:': 'Direct'}, {'Space / Unit:': 'T03', 'Space Subtype:': 'Entertainment, Retail', 'RSF:': '27365', 'Days on Market:': 

This last cell is for filtering and outputing on in a table format, it also can load from ```.JSON``` file

In [ ]:

datajson = json.load(open("data.json"))

# Normalize the datajson
#datajson = pd.json_normalize(datajson.values(),max_level=0)



df = pd.DataFrame.from_dict(datajson, orient='index')
print("Keywords for filtering: ",df.columns)

# save all the info from the scrape on a table
df.to_excel("full_scrape.xlsx") 


# This part is for the desired filtering
# 
myprops = ['link','Property Type:','Subtype:',"price",'Date Listed:','Last Updated:','names','phones','licenses']

df_filtered = df[myprops]
df_filtered.to_excel("filtered_scrape.xlsx")
print("data saved using the columns: ")
for prop in myprops:
    print(prop)


# this part save the table space by space
by_spaces = df_filtered.explode("spaces")
by_spaces = pd.concat([by_spaces.drop(['spaces'], axis=1), by_spaces['spaces'].apply(pd.Series)], axis=1)
by_spaces.to_excel("by_spaces.xlsx")
    



Keywords for filtering:  Index(['link', 'address', 'price', 'Date Listed:', 'Last Updated:',
       'Property Type:', 'Subtype:', 'Tenancy:', 'Total Building SQFT:',
       'Minimum Divisible SQFT:', 'Max Contiguous SQFT:', 'Vacant SQFT:',
       'Lot Size (sq ft):', 'Buildings:', 'County:', 'spaces', 'names',
       'phones', 'licenses', 'Class:', 'Total Building Suites:', 'Land Acres:',
       'Lot Size (acres):', 'Year Renovated:', 'Stories:', 'Ceiling Height:',
       'Loading Docks:', 'Total Parking Spaces:', 'Drive Ins:',
       'Cross Street:', 'Zoning:', 'APN:', 'Year Built:', 'Elevators:',
       'Submarket:', 'Submarket Cluster:', 'Taxes (cost per SQFT):', 'Power:',
       'Frontage:', 'Number Of Elevators:', 'Building FAR:',
       'Dock High Doors:', 'Collection Street:', 'Land SQFT:',
       'Elevator Type:'],
      dtype='object')
data saved using the columns: 
link
Property Type:
Subtype:
price
Date Listed:
Last Updated:
names
phones
licenses


KeyError: 'spaces'